In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [25]:
# ===============================
# 📌 Block 1: Install & Import Libraries
# ===============================
!pip install albumentations==1.4.3
!pip install torchsummary

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary


In [26]:
# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

CUDA Available? True


In [27]:
# ===============================
# 📌 Block 2: Define Albumentations for CIFAR-10
# ===============================
# CIFAR-10 mean & std
cifar10_mean = (0.4914, 0.4822, 0.4465)
cifar10_std = (0.2023, 0.1994, 0.2010)

train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.CoarseDropout(max_holes=1, max_height=16, max_width=16,
                    min_holes=1, min_height=16, min_width=16,
                    fill_value=cifar10_mean, mask_fill_value=None, p=0.5),
    A.Normalize(mean=cifar10_mean, std=cifar10_std),
    ToTensorV2()
])

test_transforms = A.Compose([
    A.Normalize(mean=cifar10_mean, std=cifar10_std),
    ToTensorV2()
])

# Custom Dataset wrapper for Albumentations
from torchvision.datasets import CIFAR10

class CIFAR10_Augmented(CIFAR10):
    def __init__(self, root, train=True, transform=None, download=False):
        super().__init__(root=root, train=train, download=download)
        self.transform = transform

    def __getitem__(self, idx):
        image, label = self.data[idx], self.targets[idx]
        if self.transform:
            image = self.transform(image=image)['image']
        return image, label


In [28]:
# ===============================
# 📌 Block 3: Load Dataset & Dataloaders
# ===============================
train_dataset = CIFAR10_Augmented(root='./data', train=True, transform=train_transforms, download=True)
test_dataset = CIFAR10_Augmented(root='./data', train=False, transform=test_transforms, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

print(f"Train: {len(train_dataset)} | Test: {len(test_dataset)}")


Train: 50000 | Test: 10000


In [29]:
# ===============================
# 📌 Block 4: Define the Model
# ===============================
class CIFARNet(nn.Module):
    def __init__(self):
        super(CIFARNet, self).__init__()

        # C1 - Standard Conv i/p RF =1
        #### BLOCK 1 #####
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1, bias=False)    #RF = 3
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, padding=1, bias= False)  ## RF = 5
        self.bn2 = nn.BatchNorm2d(16)
        #self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1, bias= False)  ## RF = 7
        #self.bn3 = nn.BatchNorm2d(32)
        self.depthwise1 = nn.Conv2d(16, 16, kernel_size=3, padding=1, groups=16, bias=False) # RF = 7
        self.pointwise1 = nn.Conv2d(16, 32, kernel_size=1, bias=False)
        self.bn4 = nn.BatchNorm2d(32)


        self.dilatedconv1 = nn.Conv2d(32, 64, kernel_size=3, padding=1, dilation=2, bias=False) # RF = 11 ( EDGEs & Gradients)
        self.bn5 = nn.BatchNorm2d(64)   #### Dilated Kernel
        

        self.antman1 = nn.Conv2d(64, 32, kernel_size=1, padding =0)




        ### Block 2 ######
        ### Textures & patterns ####

        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1, bias= False)  ## RF = 13
        self.bn6 = nn.BatchNorm2d(32)
        



        self.depthwise2 = nn.Conv2d(32, 32, kernel_size=3, padding=1, groups=32, bias=False) # RF = 17
        self.pointwise2 = nn.Conv2d(32, 64, kernel_size=1, bias=False)
        self.bn7 = nn.BatchNorm2d(64)
        

        #self.antman2 = nn.Conv2d(64, 32, kernel_size=1, padding =1)


        #self.depthwise3 = nn.Conv2d(16, 16, kernel_size=3, padding=1, groups=32, bias=False)
        #self.pointwise3 = nn.Conv2d(16, 32, kernel_size=1, bias=False)
        #self.bn8 = nn.BatchNorm2d(32)

        self.dilatedconv2 = nn.Conv2d(64, 64, kernel_size=3, padding=2, dilation=2, bias=False) # RF = 21
        self.bn8 = nn.BatchNorm2d(64)

        self.antman2 = nn.Conv2d(64, 32, kernel_size=1, padding =0)






        #### Block 3 ######
        #### Part of Objects ####

        self.depthwise3 = nn.Conv2d(32, 32, kernel_size=3, padding=1, groups=32, bias=False) #RF == 23
        self.pointwise3 = nn.Conv2d(32, 32, kernel_size=1, bias=False)
        self.bn9 = nn.BatchNorm2d(32)
        

        self.dilatedconv3 = nn.Conv2d(32, 50, kernel_size=3, padding=2, dilation=2, bias=False) # RF = 27
        self.bn10 = nn.BatchNorm2d(50)
        #self.antman2 = nn.Conv2d(80, 64, kernel_size=1, padding =1) # RF = 13

        self.conv5 = nn.Conv2d(50, 80, kernel_size=3, stride=2, padding=1, bias=False) ## RF= 29
        self.bn11 = nn.BatchNorm2d(80)
        self.dropout5 = nn.Dropout(0.05)

        self.antman3 = nn.Conv2d(80, 32, kernel_size=1, padding =0)

        self.depthwise4 = nn.Conv2d(32, 32, kernel_size=3, padding=1, groups=32, bias=False) #RF == 33
        self.pointwise4 = nn.Conv2d(32, 32, kernel_size=1, bias=False)
        self.bn12 = nn.BatchNorm2d(32)


       # self.conv4 = nn.Conv2d(64, 80, kernel_size=3, padding=2, dilation=2, bias=False)
       # self.bn8 = nn.BatchNorm2d(80)
       # self.antman3 = nn.Conv2d(80, 64, kernel_size=1, padding =1)




        ### Block 4 #######
        ### Objects #######

        self.conv6 = nn.Conv2d(32, 45, kernel_size=3,stride=2, padding=1, bias=False)    #RF = 35
        self.bn13 = nn.BatchNorm2d(45)
        self.dropout7 = nn.Dropout(0.05)

        self.depthwise5 = nn.Conv2d(45, 45, kernel_size=3, padding=1, groups=45, bias=False) #RF == 39
        self.pointwise5 = nn.Conv2d(45, 55, kernel_size=1, bias=False)
        self.bn14 = nn.BatchNorm2d(55)
        


        self.dilatedconv4 = nn.Conv2d(55, 55, kernel_size=3, padding=2, dilation=2, bias=False) # RF = 43
        self.bn15 = nn.BatchNorm2d(55)



        self.conv7 = nn.Conv2d(55, 80, kernel_size=3, padding=1, bias=False) ## RF == 45

       

        # Global Average Pooling
        self.gap = nn.AdaptiveAvgPool2d(1)

        # Final Fully Connected Layer
        self.fc = nn.Linear(80, 10, bias=False)

    def forward(self, x):

       ### BLOCK 1 ####
        x = F.relu(self.bn1((self.conv1(x))))
        x = F.relu(self.bn2((self.conv2(x))))
        x = F.relu(self.bn4((self.pointwise1(self.depthwise1(x)))))
        x = F.relu(self.bn5((self.dilatedconv1(x))))
        x = self.antman1(x)

        ### BLOCK2 ####

        x = F.relu(self.bn6((self.conv3(x))))
        x = F.relu(self.bn7((self.pointwise2(self.depthwise2(x)))))
        x = F.relu(self.bn8((self.dilatedconv2(x))))
        x = self.antman2(x)
       # x = F.relu(self.bn6((self.pointwise2(self.depthwise2(x))) ))


        ## BLOCK3 #####

        x = F.relu(self.bn9((self.pointwise3(self.depthwise3(x)))))
        x = F.relu(self.bn10((self.dilatedconv3(x))))
        x = F.relu(self.bn11((self.conv5(x)))) # Removed the extra call to conv5 and directly applied relu and batchnorm
        x = self.dropout5(x)
        x = self.antman3(x)
        x = F.relu(self.bn12((self.pointwise4(self.depthwise4(x)))))
        #x = self.dropout5(x)


        ### BLOCK 4 ####

        x = F.relu(self.bn13((self.conv6(x))))
        x = self.dropout7(x)
        x = F.relu(self.bn14((self.pointwise5(self.depthwise5(x)))))
        x = F.relu(self.bn15((self.dilatedconv4(x))))
        x = self.conv7(x)



        # Global Average Pooling
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = CIFARNet().cuda()
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
            Conv2d-3           [-1, 16, 32, 32]           2,304
       BatchNorm2d-4           [-1, 16, 32, 32]              32
            Conv2d-5           [-1, 16, 32, 32]             144
            Conv2d-6           [-1, 32, 32, 32]             512
       BatchNorm2d-7           [-1, 32, 32, 32]              64
            Conv2d-8           [-1, 64, 30, 30]          18,432
       BatchNorm2d-9           [-1, 64, 30, 30]             128
           Conv2d-10           [-1, 32, 30, 30]           2,080
           Conv2d-11           [-1, 32, 15, 15]           9,216
      BatchNorm2d-12           [-1, 32, 15, 15]              64
           Conv2d-13           [-1, 32, 15, 15]             288
           Conv2d-14           [-1, 64,

In [30]:
# ===============================
# 📌 Block 5: Training & Testing Functions
# ===============================
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    total_loss, correct = 0, 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()

    print(f"Epoch {epoch} Train Loss: {total_loss/len(train_loader):.4f} | Acc: {100.*correct/len(train_loader.dataset):.2f}%")

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()

    acc = 100.*correct/len(test_loader.dataset)
    print(f"Test Loss: {test_loss/len(test_loader):.4f} | Acc: {acc:.2f}%")
    return acc


In [31]:
# ===============================
# 📌 Block 6: Run Training
# ===============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

best_acc = 0
for epoch in range(1, 65):  # train for up to 65 epochs
    train(model, device, train_loader, optimizer, criterion, epoch)
    acc = test(model, device, test_loader, criterion)

    if acc > best_acc:
        best_acc = acc

print("Best Accuracy:", best_acc)


Epoch 1 Train Loss: 1.6722 | Acc: 36.67%
Test Loss: 1.3992 | Acc: 49.04%
Epoch 2 Train Loss: 1.3059 | Acc: 51.97%
Test Loss: 1.2405 | Acc: 55.57%
Epoch 3 Train Loss: 1.1597 | Acc: 58.00%
Test Loss: 1.0348 | Acc: 63.67%
Epoch 4 Train Loss: 1.0608 | Acc: 61.82%
Test Loss: 0.8835 | Acc: 68.40%
Epoch 5 Train Loss: 1.0021 | Acc: 64.35%
Test Loss: 1.0171 | Acc: 64.32%
Epoch 6 Train Loss: 0.9502 | Acc: 66.20%
Test Loss: 0.8537 | Acc: 69.55%
Epoch 7 Train Loss: 0.9050 | Acc: 68.09%
Test Loss: 0.8057 | Acc: 71.89%
Epoch 8 Train Loss: 0.8715 | Acc: 68.98%
Test Loss: 0.7406 | Acc: 74.39%
Epoch 9 Train Loss: 0.8414 | Acc: 70.08%
Test Loss: 0.7047 | Acc: 75.21%
Epoch 10 Train Loss: 0.8174 | Acc: 71.13%
Test Loss: 0.6874 | Acc: 76.04%
Epoch 11 Train Loss: 0.7960 | Acc: 71.90%
Test Loss: 0.6614 | Acc: 77.10%
Epoch 12 Train Loss: 0.7737 | Acc: 72.84%
Test Loss: 0.7327 | Acc: 75.55%
Epoch 13 Train Loss: 0.7627 | Acc: 73.31%
Test Loss: 0.6088 | Acc: 79.15%
Epoch 14 Train Loss: 0.7414 | Acc: 73.99%
Test 